In [41]:
# Library & Constants

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Literal

from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import OneHotEncoder

from xgboost import XGBClassifier

# Options

XGB_FEATER_SELECTION = False
BINARY_CLASSIFICATION = True

# Paths

DATASET_DIR = './data'
MODEL_DIR = './models'


# Datsets Info

BENIGN_LABEL = ['Benign']
MALICIUS_LABELS = {
    "Web_Attack": ["Web_XSS", "Web_SQL_Injection", "Web_Brute_Force"],
    "Dos_Attack": ["DoS_Slowhttptest", "DoS_GoldenEye", "DDoS LOIT", "DoS_Hulk", "DoS_Slowloris"],
    "Network_Attack": ["Port_Scan", "Heartbleed"],
    "Botnet": ["Botnet_ARES"],
    "Service_Attack": ["SSH-Patator", "FTP-Patator"]
}
LABELS = BENIGN_LABEL + list(MALICIUS_LABELS.keys())

# Dataset
Prima di procedere con la modellazione del dataset e alla creazione del modello, procediamo con il capire come i dati sono strutturati, cosa rappresentano e quali modifiche rapportare.

In [42]:
# Load Data

def load_data(datasets_dir: str) -> pd.DataFrame:
    """
    Load all datasets from a directory into a single DataFrame

    :param str datasets_dir: directory containing the datasets
    :return: pd.DataFrame containing all datasets
    """

    data = []
    for file in os.listdir(datasets_dir):
        if file.endswith('.csv'):
            data.append(pd.read_csv(os.path.join(datasets_dir, file)))

    return pd.concat(data, ignore_index=True)


dataset = load_data(DATASET_DIR)

## Data Exploration

Iniziamo l’analisi osservando la quantità di dati disponibili nel nostro dataset, BCCC-CIC-IDS2017, che comprende un totale impressionante di 2.438.052 record. Questi dati sono stati generati utilizzando NTLFlowLyzer, in contrasto con la versione precedente del dataset (CIC-IDS2017), che aveva invece adottato lo strumento CICFlowMeter per l'estrazione.

Per approfondire e comprendere meglio i dati a disposizione, esploriamoli graficamente per analizzarne la distribuzione e identificarne eventuali pattern significativi.

In [ ]:
# Plot Data

def plot_attack_distribution(dataset: pd.DataFrame):
    """
    Generates a bar chart to visualize the distribution of attack labels in the given dataset.

    :param pd.DataFrame dataset: The dataset containing the 'label' column to generate the pie chart from.
    """

    with plt.style.context('dark_background'):
        attack_data = dataset[~dataset['label'].isin(BENIGN_LABEL)]['label'].value_counts()
        
        fig, ax = plt.subplots(figsize=(26, 10))
        ax.bar(attack_data.index, attack_data.values, width=0.3)
        plt.show()

    plt.close()
    

def plot_distribution(dataset: pd.DataFrame):
    """
    Generates a pie chart to visualize the distribution of 'Benign' and 'Attack' labels in the given dataset.

    :param pd.DataFrame dataset: The dataset containing the 'label' column to generate the pie chart from.
    """

    with plt.style.context('dark_background'):
        data = {
            'Benign': dataset['label'].isin(BENIGN_LABEL).sum(),
            'Malicius': dataset[~dataset['label'].isin(BENIGN_LABEL)]['label'].value_counts().sum()
        }
        plt.figure(figsize=(10, 10))
        plt.pie(data.values(), labels=data.keys(), autopct='%1.1f%%', startangle=140, colors=['#66b3ff', '#D4FCC3'])
        plt.axis('equal')
        plt.show()

    plt.close()

plot_distribution(dataset)
plot_attack_distribution(dataset)

Dall'analisi dei dati emergono due osservazioni principali:
1. Il traffico normale è significativamente più rappresentato rispetto al traffico malevolo. Questo squilibrio può essere affrontato in modo intuitivo attraverso l’applicazione di tecniche di __undersampling__ sui dati relativi al traffico normale.
2. La distribuzione dei dati degli attacchi non è uniforme.

Concentrandoci sul secondo punto, l’alto sbilanciamento dei dati può influire negativamente sulle prestazioni del modello, aumentando il rischio di errori di predizione, soprattutto per le classi meno rappresentate. Per mitigare questo problema, è possibile adottare diversi approcci, ognuno con vantaggi e svantaggi:

- Escludere attacchi con pochi dati
- Applicare tecniche di undersampling e oversampling(undersampling).
- Focalizzarsi su un singolo tipo di attacco


_Per semplicità, procediamo con l'under sampling ed escludiamo il problema della distribuzione degli attacchi etichettando i dati solo con "Benign" e "Malicious"_

## Data Preparation

In [44]:
# Data Cleaning
from typing import List

def clean_data(dataset: pd.DataFrame, columns_to_remove: List[str]) -> pd.DataFrame:
    """
    Cleans the given dataset.

    :param pd.DataFrame dataset: The dataset to clean.
    :param List[str] columns_to_remove: The columns to remove from the dataset.
    :return: The cleaned dataset.
    """

    dataset = dataset.dropna()
    dataset = dataset.drop_duplicates()

    float_cols = dataset.select_dtypes(include=['float']).columns
    dataset[float_cols] = dataset[float_cols].round(4)

    dataset = dataset.drop(columns=columns_to_remove)

    return dataset


columns_to_remove = ["flow_id", "src_ip", "dst_ip", "src_port", "timestamp"]
dataset = clean_data(dataset, columns_to_remove)

In [45]:
# Data Labeling

def binary_labeling(dataset: pd.DataFrame) -> pd.DataFrame:
    """
    Converts the 'label' column in the given dataset to binary labels.

    :param pd.DataFrame dataset: The dataset to convert.
    :return: The dataset with binary labels.
    """

    dataset['label'] = dataset['label'].apply(lambda x: 'Benign' if x in BENIGN_LABEL else 'Malicius')

    return dataset

def multiclass_labeling(dataset: pd.DataFrame, mapping_label: dict) -> pd.DataFrame:
    """
    Converts the 'label' column in the given dataset to multiclass labels.

    :param pd.DataFrame dataset: The dataset to convert.
    :param dict mapping_label: The mapping of labels to convert to.
    :return: The dataset with multiclass labels.
    """

    dataset['label'] = dataset['label'].map(mapping_label)
    return dataset


if BINARY_CLASSIFICATION:
    dataset = binary_labeling(dataset)
else:
    group_map = {attack: group for group, attacks in MALICIUS_LABELS.items() for attack in attacks}
    group_map.update({BENIGN_LABEL[0]: BENIGN_LABEL[0]})
    dataset = multiclass_labeling(dataset, group_map)

In [46]:
# Data Balancing

def balance_data(dataset: pd.DataFrame) -> pd.DataFrame:
    """
    Balances the given dataset by upsampling the minority class.

    :param pd.DataFrame dataset: The dataset to balance.
    :return: The balanced dataset.
    """

    benign_data = dataset[dataset['label'].isin(BENIGN_LABEL)]
    malicious_data = dataset[~dataset['label'].isin(BENIGN_LABEL)]

    malicious_data = resample(malicious_data, replace=True, n_samples=len(benign_data), random_state=42)
    dataset = pd.concat([benign_data, malicious_data])

    return dataset

dataset = balance_data(dataset)

In [ ]:
# Data Plotting after Data Cleaning and Unbalancing

plot_attack_distribution(dataset)
plot_distribution(dataset)

In [48]:
# Splitting Features and Labels
from typing import Tuple


def extract_feature_and_target(dataset: pd.DataFrame) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Extracts the feature and target columns from the given dataset.

    :param pd.DataFrame dataset: The dataset to extract the feature and target columns from.
    :return: The feature and target columns.
    """

    return dataset.drop(columns=['label']), dataset['label']

X, y = extract_feature_and_target(dataset)

del dataset

In [49]:
# One Hot Encoding
from sklearn.preprocessing import OneHotEncoder

def one_hot_encoding(X: pd.DataFrame, y: pd.DataFrame) -> pd.DataFrame:
    """
    Applies one-hot encoding to all columns of type "object" in a pandas DataFrame.

    :param pd.DataFrame df: The DataFrame to apply one-hot encoding to.
    :return: The DataFrame with one-hot encoding applied.
    """

    object_columns = X.select_dtypes(include=['object']).columns

    encoder = OneHotEncoder(sparse_output=False)

    encoded_array = encoder.fit_transform(X[object_columns])
    encoded_columns = encoder.get_feature_names_out(object_columns)
    encoded_df = pd.DataFrame(encoded_array, columns=encoded_columns)
    X = pd.concat(
        [X.drop(columns=object_columns).reset_index(drop=True), encoded_df.reset_index(drop=True)],
        axis=1
    )

    return X, y

X, y = one_hot_encoding(X, y)

In [ ]:
# Data Preprocessing
from typing import Tuple

def normalize_data(dataset: pd.DataFrame) -> pd.DataFrame:
    """
    Normalizes the given dataset using Min-Max scaling.

    :param pd.DataFrame dataset: The dataset to normalize.
    :return: The normalized dataset.
    """

    scaler = MinMaxScaler()
    normalized_array = scaler.fit_transform(dataset)

    normalized_df = pd.DataFrame(normalized_array, columns=dataset.columns, index=dataset.index)
    return normalized_df

X = normalize_data(X)

In [ ]:
# Feature Selection

from typing import Tuple

def select_features(dataset: pd.DataFrame, target: pd.Series, threshold: float = 0.01, using_xgb: bool = True) -> pd.DataFrame:
    """
    Selects the most important features from the given dataset using an XGBoost classifier.

    :param pd.DataFrame dataset: The dataset to select features from.
    :param pd.Series target: The target column.
    :param float threshold: The threshold to select features.
    :param bool xgb: Whether to use an XGBoost classifier or RandomForest.
    :return: The selected features.
    """

    if using_xgb:
        clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
    else:
        clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(dataset, target)
    
    sfm = SelectFromModel(clf, threshold=threshold, prefit=True)    
    
    return dataset.columns[sfm.get_support()]


features = select_features(X, y, using_xgb=XGB_FEATER_SELECTION)

print(f"Selected features with threshold >= 0.01:")
for feature in features:
    print(feature)

X = X[features]

# Modelling
Procederemo con lo sviluppo del modello facendo riferimento progetto github [_"Efficient-CNN-BiLSTMs"_](https://github.com/jayxsinha/Efficient-CNN-BiLSTM-for-Network-IDS/) per la sua struttura

In [ ]:
# Data Splitting

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

del X, y

## Model Desing

I layer di questo modello sono i seguenti:
- CNN: Utile per apprendere le relazioni più nascoste tra le feature
- 2 BiLSTMs: Per poter capire come le feature hanno effetto su lungo raggio

In [ ]:
# Model Design
from keras.models import Sequential
from keras.layers import Convolution1D, MaxPooling1D, BatchNormalization
from keras.layers import Bidirectional, LSTM, Reshape, Dropout, Dense, Activation

def build_model(input_shape=(122, 1)):
    model = Sequential()
    
    # 1D Convolutional layer
    model.add(Convolution1D(64, kernel_size=122, padding="same", activation="relu", input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=5))
    model.add(BatchNormalization())
    
    # Bidirectional LSTM layer
    model.add(Bidirectional(LSTM(64, return_sequences=False)))
    
    # Reshape layer
    model.add(Reshape((128, 1), input_shape=(128,)))
    model.add(MaxPooling1D(pool_size=5))
    model.add(BatchNormalization())
    
    # Second Bidirectional LSTM layer
    model.add(Bidirectional(LSTM(128, return_sequences=False)))
    
    # Dropout layer
    model.add(Dropout(0.5))
    
    # Dense output layer with sigmoid activation
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

model = build_model(input_shape=(len(features), 1))

In [ ]:
# Model Training

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold

def train_model(model, X_train, y_train, batch_size=32, epochs=10, validation_data=None):
    """
    Trains the given model.

    :param keras.models.Sequential model: The model to train.
    :param np.ndarray X_train: The training features.
    :param np.ndarray y_train: The training target.
    :param int batch_size: The batch size for training.
    :param int epochs: The number of epochs for training.
    :param Tuple[np.ndarray, np.ndarray] validation_data: The validation features and target.
    :return: The trained model and training history.
    """

    history = model.fit(
        X_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=validation_data
    )
    return model, history


model, history = train_model(model, X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
# Training Evaluation

def plot_training_metrics(history) -> plt.Figure:
    """
    Plots training and validation metrics from the history object.

    :param history: The training history object.
    :return: Matplotlib figure of the training metrics.
    """
    print("Generating training metrics plot...")
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))

    # Accuracy Plot
    train_acc = history.history.get('accuracy', None)
    val_acc = history.history.get('val_accuracy', None)
    if train_acc and val_acc:
        ax[0].plot(train_acc, label='Training Accuracy', marker='o')
        ax[0].plot(val_acc, label='Validation Accuracy', marker='x')
        ax[0].set_title('Training and Validation Accuracy per Epoch')
        ax[0].set_xlabel('Epochs')
        ax[0].set_ylabel('Accuracy')
        ax[0].legend()
        ax[0].grid(True)

    # Loss Plot
    train_loss = history.history['loss']
    val_loss = history.history.get('val_loss', None)
    ax[1].plot(train_loss, label='Training Loss', marker='o')
    if val_loss:
        ax[1].plot(val_loss, label='Validation Loss', marker='x')
    ax[1].set_title('Training and Validation Loss per Epoch')
    ax[1].set_xlabel('Epochs')
    ax[1].set_ylabel('Loss')
    ax[1].legend()
    ax[1].grid(True)

    return fig

plot_history = plot_training_metrics(history)

In [ ]:
# Evaluation Metrics
from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score, f1_score
import seaborn as sns

def plot_evaluation_metrics(y_test, y_pred, y_pred_probs) -> plt.Figure:
    """
    Plots evaluation metrics including confusion matrix and ROC curve.

    :param y_test: True labels.
    :param y_pred: Predicted labels.
    :param y_pred_probs: Predicted probabilities.
    :return: Matplotlib figure of the evaluation metrics.
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    cm = confusion_matrix(y_test, y_pred)
    labels = ["Benign", "Malicious"]
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels, ax=axes[0])
    axes[0].set_xlabel("Predicted")
    axes[0].set_ylabel("Original")
    axes[0].set_title("Confusion Matrix")

    fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
    roc_auc = auc(fpr, tpr)
    axes[1].plot(fpr, tpr, label=f"ROC curve (AUC = {roc_auc:.2f})", color="darkorange")
    axes[1].plot([0, 1], [0, 1], color="navy", linestyle="--")
    axes[1].set_xlabel("False Positive Rate")
    axes[1].set_ylabel("True Positive Rate")
    axes[1].set_title("Receiver Operating Characteristic")
    axes[1].legend(loc="lower right")

def evaluate_model(model, X_test, y_test) -> plt.Figure:
    """
    Evaluates the model and generates plots for evaluation metrics.

    :param model: Trained model.
    :param X_test: Test features.
    :param y_test: Test labels.
    :return: Matplotlib figure containing the evaluation metrics.
    """
    y_pred_probs = model.predict(X_test).ravel()
    y_pred = (y_pred_probs >= 0.5).astype(int)

    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"Accuracy: {accuracy:.2f}")
    print(f"F1 Score: {f1:.2f}")

    return plot_evaluation_metrics(y_test, y_pred, y_pred_probs)


plot_cm_and_roc = evaluate_model(model, X_test, y_test)

In [ ]:
# Save Model

def save_model(model, path: str = MODEL_DIR, *plots: plt.Figure):
    """
    Saves the given model to the specified path.

    :param keras.models.Sequential model: The model to save.
    :param plt.Figure plot: The plot to save.
    :param str path: The path to save the model to.
    """
    
    if not os.path.exists(path):
        os.makedirs(path)

    version = len(os.listdir(path)) + 1
    model_dir = os.path.join(path, f"v{version}")

    os.makedirs(model_dir)

    model.save(os.path.join(model_dir, "model.keras"))
    for i, plot in enumerate(plots):
        plot.savefig(os.path.join(model_dir, f"plot_{i}.png"))


save_model(model, plot_cm_and_roc, plot_history)
